# Identification vs Detection: Visualizing differences on simplex

In [2]:
import sys
import itertools
import collections
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt

from utils.visualization import plot_pdf_triplex, plot_prob_triplex, reliability_plot, ECE_plot
from utils.ops import onehot_encode

In [3]:
%matplotlib inline

## Generate mesh of probabilities

In [4]:
%%time
indep_probs = np.array([np.array(tup) for tup in itertools.product(np.linspace(0, 1, num=100), repeat=2)])
indep_probs = indep_probs[np.where(np.sum(indep_probs, axis=1)<=1)]
probs = np.hstack((indep_probs, 1.-np.sum(indep_probs, axis=1, keepdims=True)))

target = onehot_encode(np.argmax(probs, axis=1))

CPU times: user 9.28 ms, sys: 4.25 ms, total: 13.5 ms
Wall time: 13.5 ms


## Compute the relative log-likelihoods

In [5]:
RLL = np.log(probs + 1e-7)
# Start asumming flat prior
priors = np.zeros(3) + 1./3

## Identification problem: 
Choose $t$ such that for any $i \neq t$ the resulting log-likelihood-ratio $\lambda^{t}_{i} = \lambda_{t} - \lambda_{i}$ is greater than the threshold: $\theta^{t}_{i} = -log(\pi_{t}) + log(\pi_{i})$, where $\{\lambda_i\}$ are the relative-log-likelihoods and $\{\pi_i\}$ is the prior probability distribution.

This corresponds to select the class with the highest posterior probability. Assuming a flat prior this is done by simply selecting the maximum over the classifier output.

*Note:*
The evaluated hypothesis are mutually exclusive, this is incompatible with the decision rule when 2 hypothesis have the same posterior probability and this is maximum. To handle this we take only the first encountered one.

**Naive approach:**
Start with the set of all possible classes as identified and check for every $t$ and $i$, whenever the condition $\lambda^{t}_{i} \geq \theta^{t}_{i}$ is not met we remove class $t$ from the set of identified classes.

In [35]:
identified = [set([0, 1, 2]) for _ in range(RLL.shape[0])]

for t in range(3):
    for i in range(3):
        if t==i:
            continue
        llr = RLL[:, t] - RLL[:, i]
        th = -np.log(priors[t]) + np.log(priors[i])
        for k in range(len(identified)):
            if llr[k] < th:
                identified[k].discard(t)
                
identified_naive = np.array([list(k)[0] for k in identified])

**Direct approach:** Having already computed the posterior probabilities we can select the class with the highest. Since the logarithm is a monotonically increasing function the analysis can be perform on the *log* domain.

In [36]:
posterior_RLL = RLL + np.log(priors)
identified = np.argmax(posterior_RLL, axis=1)

Check whether both methods agree:

In [38]:
print('Number of disagreements: {}'.format(np.sum(np.abs(identified - identified_naive))))

Number of disagreements: 0


## Detection problem:
The detection problem poses a harder constraint in order to recognize a given target, this is: that the posterior probability of the target is above 0.5